https://stackoverflow.com/questions/18390150/pls-da-algorithm-in-python

In [2]:
import numpy as np
import pandas as pd
from sklearn.cross_decomposition import PLSRegression
from sklearn.preprocessing import StandardScaler
import plotly.express as px

# サンプルデータの生成
np.random.seed(42)
X = np.random.randn(200, 10)  # 200サンプル、10特徴量
y = np.random.randint(0, 5, 200)  # 5クラスのラベル

# データの標準化
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# PLS回帰モデルの構築
n_components = 3  # 主成分の数
pls = PLSRegression(n_components=n_components)
pls.fit(X_scaled, y)

# 主成分スコアの取得
X_scores = pls.x_scores_

# データフレームの作成
df = pd.DataFrame(
    {"PC1": X_scores[:, 0], "PC2": X_scores[:, 1], "PC3": X_scores[:, 2], "Label": y}
)

# 3次元散布図の作成（マーカーサイズを小さく設定）
fig = px.scatter_3d(
    df,
    x="PC1",
    y="PC2",
    z="PC3",
    color="Label",
    title="PLS Regression: First 3 Principal Components",
    labels={
        "PC1": "First Principal Component",
        "PC2": "Second Principal Component",
        "PC3": "Third Principal Component",
        "Label": "Class Label",
    },
    color_continuous_scale=px.colors.qualitative.Set1,
    size_max=5,
)  # マーカーの最大サイズを小さく設定

# マーカーサイズをさらに調整
fig.update_traces(marker=dict(size=3))  # すべてのマーカーのサイズを3に設定

# レイアウトの調整
fig.update_layout(
    scene=dict(xaxis_title="PC1", yaxis_title="PC2", zaxis_title="PC3"),
    width=900,
    height=700,
    margin=dict(r=20, b=10, l=10, t=40),
)

# グラフの表示
fig.show()